In [1]:
%matplotlib inline 
import numpy as np

import arrayfire as af
from scipy import ndimage
import matplotlib.pyplot as plt
from LearnAF import *
import tqdm
import pandas as pd

af.set_backend('cpu')

DF = pd.read_csv('/home/narsi/Downloads/train.csv')
Data = np.asarray(DF, dtype = np.float32)/255.0
Data = Data[1:]
labels = np.asarray(DF['label'], dtype = np.float32)
classes = np.asarray(to_categorical(labels), dtype = np.float32)

# initialize weights randomly with mean 0
syn0 = np.array(2*np.random.random((Data.shape[1],64)) - 1, dtype = np.float32)
W1 = Variable(af.np_to_af_array(syn0),name='W1')
b1 = Variable(af.constant(0,1),name='b1')

syn0 = np.array(2*np.random.random((64,16)) - 1, dtype = np.float32)
W2 = Variable(af.np_to_af_array(syn0),name='W2')
b2 = Variable(af.constant(0,1),name='b2')

syn0 = np.array(2*np.random.random((16,10)) - 1, dtype = np.float32)
W3 = Variable(af.np_to_af_array(syn0),name='W3')
b3 = Variable(af.constant(0,1),name='b3')

w = [W1,b1,W2,b2,W3,b3]

# Model

In [2]:
def learner(X,W):
    # 784 -> 64
    X1 = sigmoid(add(matmul(X,W[0]),W[1]))
    # 64 -> 16
    X2 = sigmoid(add(matmul(X1,W[2]),W[3]))
    # 16 -> 10
    YP = sigmoid(add(matmul(X2,W[4]),W[5]))
    return YP

# Place Holders

In [3]:
Xin = Constant(af.np_to_af_array(np.zeros((32,Data.shape[1]),dtype = np.float32)))
Y = Constant(af.np_to_af_array(np.zeros((32,10),dtype = np.float32)))

In [4]:
YP = learner(Xin,w)
e = mse(Y,YP)
acc = accuracy(Y,YP)
sgd = SGD(lr = 0.01,momentum=0.99)

In [6]:
for i in range(1):
    epoch_acc = []
    epoch_loss = []
    total_batchs = int(Data.shape[0]/32)
    for j in tqdm.tqdm(range(total_batchs)):
        X_np = Data[j*32:(j+1)*32,:]
        Xin = Constant(af.np_to_af_array(X_np))
        Y_np = classes[j*32:(j+1)*32,:]
        Y = Constant(af.np_to_af_array(Y_np))
        
        YP = learner(Xin,w)
        e = mse(Y,YP)
        acc = accuracy(Y,YP)
        (l,w) = sgd.update(e, w, i)
        epoch_acc.append(acc)
        epoch_loss.append(l)
        
    print('Accuracy :'+str(np.mean(epoch_acc)))
    print('Loss :'+str(np.mean(epoch_loss)))

100%|██████████| 1312/1312 [07:12<00:00,  1.58it/s]

Accuracy :0.0133265054292
Loss :0.155504283137


In [7]:
YP.eval()

arrayfire.Array()
Type: float
[32 10 1 1]
    0.1504     0.0901     0.1558     0.1059     0.1361     0.1105     0.2130     0.1872     0.1224     0.2333 
    0.3893     0.2263     0.1475     0.1284     0.2521     0.2955     0.0610     0.2050     0.0675     0.3252 
    0.3853     0.0890     0.2221     0.0484     0.1704     0.2011     0.1356     0.1192     0.0698     0.3492 
    0.3032     0.1950     0.2083     0.0730     0.2338     0.1424     0.0901     0.0673     0.0570     0.2785 
    0.2970     0.0594     0.2872     0.1004     0.0858     0.1707     0.0763     0.2488     0.0627     0.1314 
    0.3795     0.0663     0.1414     0.2606     0.1549     0.2069     0.0609     0.2193     0.1492     0.1535 
    0.1556     0.1462     0.1148     0.0747     0.1392     0.1151     0.1892     0.1525     0.1030     0.3099 
    0.4484     0.1195     0.0900     0.1049     0.0997     0.1477     0.0732     0.1164     0.0941     0.3506 
    0.1830     0.0441     0.2904     0.1677     0.1085     0.1022     

In [9]:

# input dataset
X = np.array([  [0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1] ], dtype = np.float32)
X = Constant(af.np_to_af_array(X))
# output dataset            
y = np.array([[0,0,1,1]], dtype = np.float32).T
y = Constant(af.np_to_af_array(y)) 
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

# initialize weights randomly with mean 0
syn0 = np.array(2*np.random.random((3,10)) - 1, dtype = np.float32)
W1 = Variable(af.np_to_af_array(syn0),name='W1')
b1 = Variable(af.constant(0,1),name='b1')

syn0 = np.array(2*np.random.random((10,1)) - 1, dtype = np.float32)
W2 = Variable(af.np_to_af_array(syn0),name='W2')
b2 = Variable(af.constant(0,1),name='b2')

w = [W1,b1,W2,b2]

X1 = sigmoid(add(matmul(X,W1),b1))
yp = sigmoid(add(matmul(X1,W2),b2))
e = mse(y,yp)
acc = accuracy(y,yp)
sgd = SGD(lr = 0.001,momentum=0.99)